### Recipes 5k

* [Dataset](http://www.ub.edu/cvub/recipes5k/)

* [Original Paper](https://www.researchgate.net/publication/318729535_Food_Ingredients_Recognition_through_Multi-label_Learning)

### Setup Environment:

In [1]:
import os
import pandas as pd

from src.classifiers import preprocess_data, process_labels,split_data

from src.classifiers import VQADataset
from torch.utils.data import DataLoader

from src.classifiers_cpu_metrics import train_early_fusion, train_late_fusion

In [2]:
PATH = 'Embeddings_vlm/Recipes5k/'
FILE = 'embeddings_clip.csv'

FILE_PATH = os.path.join(PATH, FILE)

## Get data

In [3]:
df = pd.read_csv(FILE_PATH)
#df.drop(columns=['image'], inplace=True)
df.head()

,questions,answers,types,split,image_embedding_0,image_embedding_1,image_embedding_2,image_embedding_3,image_embedding_4,image_embedding_5,...,text_embedding_502,text_embedding_503,text_embedding_504,text_embedding_505,text_embedding_506,text_embedding_507,text_embedding_508,text_embedding_509,text_embedding_510,text_embedding_511
0,what is using umbrellas as a central theme,sculpture,0,train,-0.002133,0.026865,-0.009941,-0.000407,-0.001535,0.043021,...,0.013656,0.006139,0.018542,-0.027902,-0.010581,0.001189,-0.018054,0.006616,0.039209,-0.052176
1,what walks toward the rope perimeter fence,elephant,0,train,-0.037488,-0.025273,0.021552,0.056967,0.010018,-0.042177,...,-0.007028,-0.002520,0.001875,-0.028708,0.016469,-0.011620,-0.010105,0.034845,-0.021513,0.029620
2,what is the color of the horses,brown,2,train,-0.050188,0.034539,0.008536,0.017248,0.027789,-0.020876,...,0.009025,0.000571,-0.022466,0.010579,-0.025592,0.034800,-0.007262,0.010370,-0.009308,0.008854
3,where is the black cat laying down,sink,3,train,-0.003502,0.007140,0.014187,0.052844,0.012376,-0.005956,...,-0.012388,0.034164,0.007408,0.029274,-0.018850,-0.007775,0.033192,0.001180,-0.041182,0.004494
4,what is the color of the character,purple,2,train,0.046207,0.050791,-0.010908,0.025887,-0.029377,-0.023032,...,-0.009319,-0.034246,0.002291,-0.004910,-0.025292,0.012616,-0.017789,-0.012675,-0.004709,0.028756


## Data Perparation

In [4]:
# Split the data
train_df, test_df = split_data(df)

# Select features and labels vectors
text_columns = [column for column in df.columns if 'text' in column] #[f'text_{i}']
image_columns = [column for column in df.columns if 'image' in column] #[f'image_{i}']
label_columns = 'class'


# Process and one-hot encode labels for training set
train_labels, mlb, train_columns = process_labels(train_df, col=label_columns)
test_labels = process_labels(test_df, col=label_columns, train_columns=train_columns)


train_dataset = VQADataset(train_df, text_columns, image_columns, label_columns, mlb, train_columns)
test_dataset = VQADataset(test_df, text_columns, image_columns, label_columns, mlb, train_columns)


train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

Train Shape: (78736, 1028)
Test Shape: (38948, 1028)


### Models

In [5]:
text_input_size = len(text_columns)
image_input_size = len(image_columns)
if label_columns == '2_way_label':
    output_size = 1
else:
    output_size = len(pd.unique(train_df[label_columns]))
multilabel = False

In [ ]:
# Train early fusion model
print("Training Early Fusion Model:")
train_early_fusion(train_loader, test_loader, text_input_size, image_input_size, output_size, num_epochs=5, multilabel=multilabel, report=True, p=0.2)

Training Early Fusion Model:
Model Memory Usage: 0.71 MB
The number of parameters of the model are: 186926


STAGE:2024-02-07 17:26:52 44107:44107 ActivityProfilerController.cpp:312] Completed Stage: Warm Up


Epoch 1/10 - Test Accuracy: 0.4370
Epoch 1/10 - Training Time: 7.56 seconds | Inference Time: 3.16 seconds
Epoch 2/10 - Test Accuracy: 0.4564
Epoch 2/10 - Training Time: 5.66 seconds | Inference Time: 2.78 seconds
Epoch 3/10 - Test Accuracy: 0.4652
Epoch 3/10 - Training Time: 6.61 seconds | Inference Time: 3.07 seconds
Epoch 4/10 - Test Accuracy: 0.4561
Epoch 4/10 - Training Time: 6.29 seconds | Inference Time: 2.99 seconds
Epoch 5/10 - Test Accuracy: 0.4768
Epoch 5/10 - Training Time: 6.76 seconds | Inference Time: 3.07 seconds
Epoch 6/10 - Test Accuracy: 0.4833
Epoch 6/10 - Training Time: 6.70 seconds | Inference Time: 3.13 seconds
Epoch 7/10 - Test Accuracy: 0.4924
Epoch 7/10 - Training Time: 6.71 seconds | Inference Time: 3.18 seconds
Epoch 8/10 - Test Accuracy: 0.4892
Epoch 8/10 - Training Time: 6.58 seconds | Inference Time: 3.23 seconds
Epoch 9/10 - Test Accuracy: 0.5029
Epoch 9/10 - Training Time: 6.56 seconds | Inference Time: 3.49 seconds
Epoch 10/10 - Test Accuracy: 0.4974
E

STAGE:2024-02-07 17:28:42 44107:44107 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2024-02-07 17:28:42 44107:44107 ActivityProfilerController.cpp:322] Completed Stage: Post Processing


In [ ]:
# Train late fusion model
print("Training Late Fusion Model:")
train_late_fusion(train_loader, test_loader, text_input_size, image_input_size, output_size, num_epochs=5, multilabel=multilabel, report=True, p=0.2)